In [1]:
import tensorflow as tf
import os
import numpy as np
from model import *
from utility import *

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

model_name = "BEGAN_for_MNIST"

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.g_bn0 = BatchNormalization(name = 'g_bn0')

    def __call__(self, z, training=False):
        with tf.variable_scope('g', reuse=self.reuse):
            fc0 = full_connection_layer(z, 7*7*512, name="fc0")
            fc0 = tf.reshape(fc0, [-1, 7, 7, 512])

            batch_size = tf.shape(fc0)[0]
            deconv0 = deconv2d_layer(fc0, [batch_size, 14, 14, 256], kernel_size=4, name="deconv0")
            deconv0 = self.g_bn0(deconv0)
            deconv0 = lrelu(deconv0, leak=0.3)

            deconv1 = deconv2d_layer(deconv0, [batch_size, 28, 28, 1], kernel_size=4, name="deconv1")
            deconv1 = tf.nn.tanh(deconv1)
            output = deconv1 

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return output

In [3]:
class Discriminator:
    def __init__(self):
        self.reuse = False
        self.d_bn0 = BatchNormalization(name="d_bn0")
        self.d_bn1 = BatchNormalization(name="d_bn1")
        
    def __call__(self, x,training=False, name=''):
        with tf.variable_scope('d', reuse=self.reuse):
            x = tf.reshape(x, [-1, 28, 28, 1])

            conv1 = conv2d_layer(x, 64, kernel_size=4, name="d_conv0")
            conv1 = tf.nn.relu(conv1)
            conv1 = tf.reshape(conv1, [-1, 64*14*14])

            fc0 = full_connection_layer(conv1, 32, name="fc0")
            fc0 = self.d_bn0(fc0)
            fc0 = tf.nn.relu(fc0)
            
            fc1 = full_connection_layer(fc0, 64*14*14, name="fc1")
            fc1 = self.d_bn1(fc1)
            fc1 = tf.nn.relu(fc1)
            fc1 = tf.reshape(fc1, [-1, 14,14,64])
            
            batch_size = tf.shape(fc1)[0]
            deconv0 = deconv2d_layer(fc1, [batch_size, 28, 28, 1], kernel_size=4, name="deconv0")
            output = tf.nn.tanh(deconv0)

            recon_error = tf.reduce_mean(tf.abs(output - x))

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')

        return output, recon_error

In [4]:
class GAN:
    def __init__(self):
        self.batch_size = 64
        self.img_size = 28
        self.z_size = 50
        
        self.epochs = 50000
        self.epoch_saveMetrics = 1000
        self.epoch_saveSampleImg = 1000
        self.epoch_saveParamter = 5000
        self.losses = {"d_loss":[], "g_loss":[], "M_value":[]}

        # unrolled counts
        self.steps = 5

        self.X_tr = tf.placeholder(tf.float32, shape=[None, self.img_size, self.img_size, 1])
        self.z = tf.placeholder(tf.float32, [None, self.z_size])
        
        self.g = Generator()
        self.d = Discriminator()
        self.Xg = self.g(self.z)

        self.k = tf.Variable(0., trainable=False)
        self.lambda_ = 1e-3
        self.gamma = 0.75

    def loss(self):
        output_tr, recon_error_tr = self.d(self.X_tr)
        output_gen, recon_error_gen = self.d(self.Xg)
        
        loss_d = recon_error_tr - self.k*recon_error_gen
        loss_g = recon_error_gen

        self.M = recon_error_tr + tf.abs(self.gamma*recon_error_tr - recon_error_gen)
        self.update_k = self.k.assign(self.k + self.lambda_*(self.gamma*recon_error_tr - recon_error_gen))

        return loss_g, loss_d

    def train(self):
        # Optimizer
        d_lr = 2e-4
        d_beta1 = 0.5
        g_lr = 2e-4
        g_beta1 = 0.5

        self.L_g, self.L_d = self.loss()

        d_opt = tf.train.AdamOptimizer(learning_rate=d_lr)
        d_train_op = d_opt.minimize(self.L_d, var_list=self.d.variables)
        g_opt = tf.train.AdamOptimizer(learning_rate=g_lr)
        g_train_op = g_opt.minimize(self.L_g, var_list=self.g.variables)

        saver = tf.train.Saver()
        
        config = tf.ConfigProto(
            gpu_options=tf.GPUOptions(
                visible_device_list= "0"
            )
        )
                
        with tf.Session(config=config) as sess:
            sess.run(tf.global_variables_initializer())

            # preparing noise vec for test
            bs = 100
            test_z = np.random.uniform(-1, 1, size=[bs, self.z_size])

            for epoch in range(self.epochs):
                # extract images for training
                X_mb, _ = mnist.train.next_batch(self.batch_size)
                X_mb = np.reshape(X_mb, [-1, 28, 28, 1])
                X_mb = (X_mb-0.5) * 2.0

                z = np.random.uniform(-1, 1, size=[self.batch_size, self.z_size])

                # train Discriminator
                _, d_loss_value = sess.run([d_train_op, self.L_d], feed_dict={
                    self.X_tr: X_mb,
                    self.z:z,
                })

                # train Generator
                _, g_loss_value = sess.run([g_train_op, self.L_g], feed_dict={
                    self.X_tr: X_mb,
                    self.z:z,
                })

                # update k
                M_value, _ = sess.run([self.M, self.update_k], feed_dict={
                    self.X_tr: X_mb,
                    self.z:z,
                })

                # append loss value for visualizing
                self.losses["d_loss"].append(np.sum(d_loss_value))
                self.losses["g_loss"].append(np.sum(g_loss_value))
                self.losses["M_value"].append(M_value)                

                # print epoch
                if epoch % 50 == 0:
                    print('epoch:{0}, d_loss:{1}, g_loss:{2}, M:value:{3} '.format(epoch, d_loss_value, g_loss_value, M_value))
                
                # visualize loss
                if epoch % self.epoch_saveMetrics == 0:
                    save_metrics(model_name, self.losses, epoch)

                # visualize generated images during training
                if epoch % self.epoch_saveSampleImg == 0:
                    img = sess.run(self.Xg, feed_dict={self.z: test_z})
                    img = (img*0.5)+0.5
                    save_imgs(model_name, img, name=str(epoch))

                # save model parameters 
                if epoch % self.epoch_saveParamter == 0:
                    dir_path = "model_" + model_name
                    if not os.path.isdir(dir_path):
                        os.makedirs(dir_path)

                    saver.save(sess, dir_path + "/" + str(epoch) + ".ckpt")

In [5]:
gan = GAN()
gan.train()

/home/ujitoko/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:57: DeprecationWarning: PyUnicode_AsEncodedObject() is deprecated; use PyUnicode_AsEncodedString() to encode from str to bytes or PyCodec_Encode() for generic encoding


epoch:0, d_loss:0.9266582727432251, g_loss:0.32139384746551514, M:value:1.317009687423706 
epoch:50, d_loss:0.4104168713092804, g_loss:0.10977968573570251, M:value:0.6099072694778442 
epoch:100, d_loss:0.28379660844802856, g_loss:0.10200047492980957, M:value:0.4004284739494324 
epoch:150, d_loss:0.2605951726436615, g_loss:0.10685686767101288, M:value:0.3577074408531189 
epoch:200, d_loss:0.2108721286058426, g_loss:0.09217820316553116, M:value:0.2827048599720001 
epoch:250, d_loss:0.2048768848180771, g_loss:0.09671444445848465, M:value:0.2698349356651306 
epoch:300, d_loss:0.20069558918476105, g_loss:0.08937438577413559, M:value:0.27021464705467224 
epoch:350, d_loss:0.2099509835243225, g_loss:0.09292273968458176, M:value:0.2831854820251465 
epoch:400, d_loss:0.15877628326416016, g_loss:0.08648265153169632, M:value:0.20050793886184692 
epoch:450, d_loss:0.19640609622001648, g_loss:0.0803074836730957, M:value:0.27206504344940186 
epoch:500, d_loss:0.16910424828529358, g_loss:0.0796014294

KeyboardInterrupt: 